In [1]:
import sys
sys.path.append(".")  

import joblib
import pickle


from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import pandas as pd
import uvicorn
import os

from swagger_for_mkdocs.transform_swagger_json import save_openapi_config
from my_logger.custom_logger import  logger
app = FastAPI()
import src.Model as Model


sys.modules['Module'] = Model

ModuleNotFoundError: No module named 'src'

In [16]:
model_name = 'model.pkl'

current_directory = os.path.dirname(os.path.realpath('scoring_script.py'))
# Construct the relative path to the model file
model_path = os.path.join(current_directory, model_name)
logger.info(f'Get model from {model_path}')
# Check if the model file exists
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file '{model_name}' not found in the current directory.")

# Load the model
with open (model_path, 'rb') as m:
    model = pickle.load(m)
print(type(model))

2024-02-06 11:36:19,998 - my_logger.custom_logger - INFO - Get model from C:\Users\bartw\Documents\cAlgo\Sources\Robots\Data_Collect_32\Data_Collect_32\deploy\model.pkl
<class 'Model.SklearnPipeline.CustomPipeline'>


In [17]:
data = {
"Datum": "1900-01-01",
  "mfi1": "2",
  "rsi1": "1",
}


new_data = pd.DataFrame.from_dict(data, orient="index").T

        # Convert Datum to datetime and other columns to numeric
new_data["Datum"] = pd.to_datetime(new_data["Datum"], dayfirst=True)
columns_to_convert = new_data.columns.difference(["Datum"])

def convert_to_numeric(column):
            return pd.to_numeric(column, errors="coerce")

new_data[columns_to_convert] = new_data[columns_to_convert].apply(
    convert_to_numeric
)

new_data = new_data[['mfi1', 'rsi1']]



In [18]:
def convert_to_numeric(column):
            return pd.to_numeric(column, errors="coerce")

new_data[columns_to_convert] = new_data[columns_to_convert].apply(
    convert_to_numeric
)

In [19]:
new_data = new_data[['mfi1', 'rsi1']]

In [20]:
new_data.loc[len(new_data)] = [4,2]

In [21]:
new_data

,mfi1,rsi1
0,2,1
1,4,2


In [23]:
model.predict_proba(new_data[['rsi1', 'mfi1']])

2024-02-06 11:36:43,650 - my_logger.custom_logger - INFO - Selecting features from Dataset
2024-02-06 11:36:43,651 - my_logger.custom_logger - INFO - Normalizing cols: ['rsi1']
2024-02-06 11:36:43,652 - my_logger.custom_logger - INFO - Normalizing cols: ['mfi1']


array([[0.51503759, 0.48496241],
       [0.51503759, 0.48496241]])

In [11]:
model.pipeline.predict(new_data)

2024-02-06 11:35:09,270 - my_logger.custom_logger - INFO - Selecting features from Dataset
2024-02-06 11:35:09,273 - my_logger.custom_logger - INFO - Normalizing cols: ['rsi1']
2024-02-06 11:35:09,276 - my_logger.custom_logger - INFO - Normalizing cols: ['mfi1']


array([0, 0])

In [12]:
model.features

['rsi1', 'mfi1']

In [13]:
hist = model.pipeline.steps[-1]

In [14]:
new_data

,mfi1,rsi1
0,2,1
1,4,2


In [ ]:
hist.get_metadata_routing

<bound method _MetadataRequester.get_metadata_routing of HistGradientBoostingClassifier(l2_regularization=1, max_bins=250, max_depth=15,
                               max_iter=25, max_leaf_nodes=60,
                               min_samples_leaf=5, random_state=2)>

In [ ]:
from sklearn.feature_selection import SelectFromModel

In [ ]:
model.pipeline

Pipeline(steps=[('FeatureSelector', FeatureSelector(features={'rsi1', 'mfi1'})),
                ('NormCols',
                 AddNormalizedColsTransformer(indicators=['rsi', 'mfi'],
                                              window=1)),
                ('StandardScaler', StandardScaler()),
                ('calibrated_classifier',
                 CalibratedClassifierCV(cv=10, ensemble=False,
                                        estimator=HistGradientBoostingClassifier(l2_regularization=1,
                                                                                 max_bins=250,
                                                                                 max_depth=15,
                                                                                 max_iter=25,
                                                                                 max_leaf_nodes=60,
                                                                                 min_samples_leaf=5,
                                                                                 random_state=2),
                                        method='isotonic'))])

In [ ]:
hist.predict_proba(new_data)

NotFittedError: This HistGradientBoostingClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.